In [58]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator
from quara.protocol.qtomography.standard.projected_linear_estimator import ProjectedLinearEstimator

from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

In [59]:
from numpy.testing import assert_allclose

In [60]:
from quara.data_analysis import data_analysis, physicality_violation_check

In [61]:
%load_ext autoreload
%autoreload 2

The autoreload extension is already loaded. To reload it, use:
  %reload_ext autoreload


# 1 ">=": True "=I": True 

In [62]:
# setup system
e_sys = ElementalSystem(0, get_comp_basis())
c_sys = CompositeSystem([e_sys])

In [63]:
m_1 = (1/2) * np.array([1, 0, 0, 1])
m_2 = (1/2) * np.array([1, 0, 0, 1])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [64]:
assert povm.is_positive_semidefinite() is True
assert povm.is_identity_sum() is True

In [69]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(proj_ineq_povm.vecs)  # expected: (array([0.5, 0. , 0. , 0.5]), array([0.5, 0. , 0. , 0.5]))

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(proj_eq_povm.vecs)  # expected: (array([0.5, 0. , 0. , 0.5]), array([0.5, 0. , 0. , 0.5]))

calc_proj_ineq_constraint():
>=0: True
(array([0.5, 0. , 0. , 0.5]), array([0.5, 0. , 0. , 0.5]))
---------------
calc_proj_eq_constraint():


/Users/tomoko/project/rcast/workspace/quara/quara/objects/povm.py:250: ComplexWarning: Casting complex values to real discards the imaginary part
  new_vec = np.array(new_vec, dtype=np.float64)


ValueError: basis is not hermitian.

In [56]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert_allclose(povm.vecs, proj_ineq_povm.vecs)

In [57]:
assert proj_eq_povm.is_identity_sum() is True
assert_allclose(povm.vecs, proj_eq_povm.vecs)

AssertionError: 

In [35]:
proj_eq_povm.matrices()

[array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j]]),
 array([[0.70710678+0.j, 0.        +0.j],
        [0.        +0.j, 0.        +0.j]])]

c_sys

# 2 ">=": False "=I": True 

In [12]:
m_1 = np.array([2., 0., 0., 0.])
m_2 = np.array([-1., 0., 0., 1.])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [13]:
assert povm.is_positive_semidefinite() is False
assert povm.is_identity_sum() is True

In [14]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(proj_ineq_povm.vecs)  # expected: != povm.vecs

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(proj_eq_povm.vecs)  # expected: != povm.vecs

calc_proj_ineq_constraint():
>=0: True
(array([2., 0., 0., 0.]), array([0., 0., 0., 1.]))
---------------
calc_proj_eq_constraint():
=I: False
(array([ 2.20710678,  0.        ,  0.        , -0.5       ]), array([-0.79289322,  0.        ,  0.        ,  0.5       ]))


In [15]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert not np.allclose(povm.vecs, proj_ineq_povm.vecs)

In [16]:
assert proj_eq_povm.is_identity_sum() is True
assert_allclose(povm.vecs, proj_eq_povm.vecs)

AssertionError: 

# 3 ">=": True "=I": False

In [17]:
m_1 = np.array([1., 0., 0., 1.])
m_2 = np.array([1., 0., 0., 1.])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [18]:
assert povm.is_positive_semidefinite() is True
assert povm.is_identity_sum() is False

In [19]:
print("calc_proj_ineq_constraint():")
proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(proj_ineq_povm.vecs)  # expected: ([1, 0, 0, 1], [1, 0, 0, 1])

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(proj_eq_povm.vecs)  # exoected: != povm.vecs

calc_proj_ineq_constraint():
>=0: True
(array([1., 0., 0., 1.]), array([1., 0., 0., 1.]))
---------------
calc_proj_eq_constraint():
=I: False
(array([0.70710678, 0.        , 0.        , 0.        ]), array([0.70710678, 0.        , 0.        , 0.        ]))


In [20]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert_allclose(povm.vecs, proj_ineq_povm.vecs)

In [21]:
assert proj_eq_povm.is_identity_sum() is True
assert_allclose(povm.vecs, proj_eq_povm.vecs)

AssertionError: 

# 4 ">=": False "=I": False

In [22]:
m_1 = np.array([1., 0., 0., 1.])
m_2 = (-1) * np.array([1., 0., 0., 1.])

vecs = [m_1, m_2]
povm = Povm(c_sys=c_sys, vecs=vecs, is_physicality_required=False)

In [23]:
assert povm.is_positive_semidefinite() is False
assert povm.is_identity_sum() is False

In [24]:
print("calc_proj_ineq_constraint():")

proj_ineq_povm = povm.calc_proj_ineq_constraint()
print(f">=0: {proj_ineq_povm.is_positive_semidefinite()}")  # expected: True

print(proj_ineq_povm.vecs)  # expected: != povm.vecs

print("---------------")

print("calc_proj_eq_constraint():")
proj_eq_povm = povm.calc_proj_eq_constraint()
print(f"=I: {proj_eq_povm.is_identity_sum()}")  # expected: True 

print(proj_eq_povm.vecs)  # expected: != povm.vecs

calc_proj_ineq_constraint():
>=0: True
(array([1., 0., 0., 1.]), array([0., 0., 0., 0.]))
---------------
calc_proj_eq_constraint():
=I: False
(array([1.70710678, 0.        , 0.        , 1.        ]), array([-0.29289322,  0.        ,  0.        , -1.        ]))


In [25]:
assert proj_ineq_povm.is_positive_semidefinite() is True
assert not np.allclose(povm.vecs, proj_ineq_povm.vecs)

In [26]:
assert proj_eq_povm.is_identity_sum() is True
assert not np.allclose(povm.vecs, proj_eq_povm.vecs)

AssertionError: 